In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
mode = "random"  # can be 'random' or 'id'
image_id = "CA214331L0344"
fid_marks = [1, 2, 3, 4]

In [17]:
import src.base.connect_to_database as ctd
import src.load.load_image as li

# create connection to database
conn = ctd.establish_connection()

# if random mode, get a random image id
if mode == "random":
    sql_string = "SELECT image_id FROM images_fid_points ""WHERE subset_n_x IS NOT NULL OR subset_e_x IS NOT NULL OR ""subset_s_x IS NOT NULL OR subset_w_x IS NOT NULL ""ORDER BY RANDOM() LIMIT 1"
    image_id = ctd.execute_sql(sql_string, conn)['image_id'].iloc[0]

# get subset information from data
sql_string = f"SELECT * FROM images_fid_points WHERE image_id='{image_id}'"
data = ctd.execute_sql(sql_string, conn).iloc[0]

# load image
image = li.load_image(image_id)

In [18]:
# Position of fid marks:
# 3 7 2
# 5   6
# 1 8 4

In [19]:
import src.fid_marks.calculate_fid_mark as cfm
import src.display.display_images as di

fid_marks = []

for key in ["ne", "nw", "se", "sw"]:

    # split the key in the two directions
    direction1 = list(key)[0]
    direction2 = list(key)[1]

    subset_width = data["subset_width"]
    subset_height = data["subset_height"]

    # get the subset bounds for both directions
    subset1_x = data[f"subset_{direction1}_x"]
    subset1_y = data[f"subset_{direction1}_y"]
    subset2_x = data[f"subset_{direction2}_x"]
    subset2_y = data[f"subset_{direction2}_y"]

    # create bounds for the subsets
    subset1 = (subset1_x, subset1_y, subset1_x + subset_width, subset1_y + subset_height)
    subset2 = (subset2_x, subset2_y, subset2_x + subset_width, subset2_y + subset_height)

    # create a list of subsets
    subsets = [subset1, subset2]


    # calculate the fid mark
    fid_mark = cfm.calculate_fid_mark(image, key, subsets, display=True)

    if fid_mark is not None:
        fid_marks.append(fid_mark)

print(fid_marks)

di.display_images(image, points=[fid_marks])


[(9764, 157), (695, 204), (9742, 9290), (650, 9249)]
